## Claude Opus SAT Solver w/ tool use
This notebook uses Anthropic's Claude Opus LLM to solve SAT problems. It uses a tool that parses the SAT problem into a structured format with the help of Pydantic, then uses Z3 to solve the SAT problem symbolically.


Run it yourself on colab:

<a href="https://colab.research.google.com/github/mbrotos/claude_sat_solver/blob/main/claude_sat_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [1]:
# Uncomment the following lines to install the dependencies and set the API key:
# !pip install anthropic z3-solver pydantic
# %env API_KEY=<your_api_key>
import os
from anthropic import Anthropic
from z3 import Solver, Bool, Or, And, Not # This is the Z3 library for SAT solving
from pydantic import BaseModel
from typing import List
api_key = os.getenv("API_KEY")

# Debug mode flag
DEBUG = True

In [2]:
# Here are some satisfiability test cases. Please add your own!
# The format is (prompt/english SAT problem, list of Z3 clauses)
test_cases = [
    (
        """
        Consider the following SAT problem in propositional logic:
        1. If it rains, the ground will be wet.
        2. If it is cloudy, it will rain.
        3. If the ground is not wet, then it is not cloudy.
        4. The ground is not wet.
        Determine if this set of statements is consistent (satisfiable) using propositional logic and the resolution method.
        """,
        [
            Or([Not(Bool('r')), Bool('w')]), # p implies q == ~p or q
            Or([Not(Bool('c')), Bool('r')]),
            Or([Bool('w'), Not(Bool('c'))]), # ~q implies p == q or ~p
            Or([Not(Bool('w'))])
        ]
    ),
    (
        """
        Analyze the satisfiability of the following propositional logic statements:
        1. If it's sunny, then it's hot.
        2. If it's hot, then we'll go to the beach.
        3. We're not going to the beach.
        4. It's sunny.
        Is this set of statements satisfiable?
        """,
        [
            Or([Not(Bool('s')), Bool('h')]),
            Or([Not(Bool('h')), Bool('b')]),
            Or([Not(Bool('b'))]),
            Or([Bool('s')])
        ]
    ),
    (
        """
        Evaluate the consistency of these logical statements:
        1. If the alarm is set, the door will be locked.
        2. If the door is locked, the windows are closed.
        3. The windows are open.
        4. The alarm is set.
        Are these statements satisfiable together?
        """,
        [
            Or([Not(Bool('a')), Bool('l')]),
            Or([Not(Bool('l')), Not(Bool('w'))]),
            Or([Bool('w')]),
            Or([Bool('a')])
        ]
    )
]

In [3]:
# Define a Pydantic model for structured output
class SATProblem(BaseModel):
    variables: List[str]
    clauses: List[List[str]]

In [4]:
# Function to parse the structured output into Z3 format using Z3's symbolic logical operators
def parse_to_z3(sat_problem: SATProblem):
    solver = Solver()
    variables = {var: Bool(var) for var in sat_problem.variables}
    
    all_clauses = []
    
    for clause in sat_problem.clauses:
        literals = []
        for literal in clause:
            if literal.startswith('~'):
                var_name = literal[1:]
                literals.append(Not(variables[var_name]))
            else:
                literals.append(variables[literal])
        all_clauses.append(Or(literals))
    
    solver.add(And(all_clauses))
    return solver

In [5]:
# Function to solve the SAT problem using Z3
def solve_sat_problem(solver: Solver):
    result = solver.check().r
    if result == 1:
        return "Satisfiable"
    elif result == -1:
        return "Unsatisfiable"
    else:
        return "Unknown"


In [6]:
tool = {
    "name": "SATParser",
    "description": "Parses SAT problems into structured format. For variables, use simple strings like 'r', 'w', 'c'. For clauses, use '~' for negation.",
    "input_schema": {
        "type": "object",
        "properties": {
            "variables": {
                "type": "array",
                "items": {"type": "string"},
                "description": "List of variables in the SAT problem"
            },
            "clauses": {
                "type": "array",
                "items": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "List of literals in the clause, use '~' for negation" # this is important for later symbolic conversion
                },
                "description": "List of clauses in the SAT problem"
            }
        },
        "required": ["variables", "clauses"]
    }
}

In [7]:
# Function to interact with Claude and solve the SAT problem
def solve_with_claude(prompt: str):
    claude = Anthropic(api_key=api_key)
    
    response = claude.messages.create(
        model="claude-3-opus-latest",
        messages=[{
            "role": "user", 
            "content": f"""Please analyze this SAT problem and use the SATParser tool to convert it into a structured format. 
            First explain your reasoning, then use the tool with the appropriate variables and clauses.
            
            {prompt}"""
        }],
        tools=[tool],
        tool_choice={"type": "auto"},
        max_tokens=500
    )
    
    if DEBUG:
        print("Claude's response:")
        for content in response.content:
            if content.type == 'text':
                print("\t" + content.text.replace("\n", "\n\t"))
            elif content.type == 'tool_use':
                print(f"\tTool use: {content.name}")
                tool_input = str(content.input).replace('\n', '\n\t')
                print(f"\tTool input: {tool_input}")
        print("\n")

    for content in response.content:
        if content.type == 'tool_use':
            tool_input = content.input
            sat_problem = SATProblem(**tool_input)
            solver = parse_to_z3(sat_problem)
            return solve_sat_problem(solver)
    
    return "No tool use found in response"

In [8]:
for i, (prompt, clauses) in enumerate(test_cases, 1):
    print("=" * 50)
    print(f"Test Case {i}:")
    print(prompt)
    result = solve_with_claude(prompt)
    print(f"Result: {result}\n")

    # Let's also check the ground truth Z3 solver provided for each test case
    if clauses:
        print(f"Ground truth Z3 clauses: {clauses}")
        gt_solver = Solver()
        gt_solver.add(And(clauses))
        print(f"Z3 result: {solve_sat_problem(gt_solver)}")
    else:
        print('No ground truth clauses provided.')

    print("=" * 50 + "\n")


Test Case 1:

        Consider the following SAT problem in propositional logic:
        1. If it rains, the ground will be wet.
        2. If it is cloudy, it will rain.
        3. If the ground is not wet, then it is not cloudy.
        4. The ground is not wet.
        Determine if this set of statements is consistent (satisfiable) using propositional logic and the resolution method.
        
Claude's response:
	<thinking>
	To analyze this SAT problem, we need to first convert the given statements into propositional logic clauses. Let's define the following variables:
	
	r: It rains
	w: The ground is wet 
	c: It is cloudy
	
	The statements can be converted as follows:
	
	1. If it rains, the ground will be wet. 
	   r → w
	   This can be written as (~r ∨ w)
	
	2. If it is cloudy, it will rain.
	   c → r 
	   This can be written as (~c ∨ r)
	
	3. If the ground is not wet, then it is not cloudy.
	   ~w → ~c
	   This can be written as (w ∨ ~c)
	
	4. The ground is not wet.
	   ~w
	
	So t